# Automated NVC Conversational Mediator
## Part 1 - proof of concept; can it work

**Purpose:** NVC uses a very standardized communication language to express feelings and needs. It is possible that this feature of NVC can be leveraged to automate some aspects of NVC conversations. The purpose of this notebook is to explore what is possible and how it may work.

**Instructions:**
Go through read and run all the cells (hit play or Shift+Enter)

# 👉 Load Packages and NVC Data Files
These are needed for the program to run

In [1]:
# Verify python and pip are running from your jupyter env (optional)
# !which python3
# !which pip3

In [2]:
# Install packages
# !pip3 install nltk, pandas, pyttsx3
# !python3 -m spacy download en

In [40]:
# Import packages
import nltk
from nltk.stem import PorterStemmer
import pattern.en as en  # I like their lemma builder

import spacy
from spacy import displacy

import pyttsx3

import pandas as pd
import numpy as np

from enum import Enum
import re, time

In [39]:
# Initialize tools
engine = pyttsx3.init()
nlp = spacy.load("en")
ps = PorterStemmer()

## Load NVC Language Data 

This is a datafile containing keywords and phrases that will be captured when speaking. These keywords/phrases will help us 

* categorize the user input (feeling, need, fofeeling, request), 
* if it is a feeling, tell us if it is a feeling that happens when our needs are unmet
* if it is a fofeeling, make some feeling guesses
* etc. (more can be developed)

First, load the datafile that contains the nvc key words

In [ ]:
# Load json datafile
kw_data = pd.read_csv("data/kw_datafile.csv", index_col='kw')

# Lowercase everything
kw_data.index = kw_data.index.str.lower()
kw_data = kw_data.apply(lambda x: x.str.lower(), axis=1)

# Check for duplicates - TODO
# kw_data[kw_data.duplicated(subset=None)]
# # kw_data.drop_duplicates()

kw_data.sample(3)

In [7]:
# Create keyword stems (for syntactical analysis later) and set the stems as the index
kw_data['kws'] = [en.lexeme(word) for word in kw_data.index]

In [8]:
kw_data['kw_base'] = [en.lemma(word) for word in kw_data.index]
kw_data = kw_data.set_index('kw_base')

In [9]:
kw_data.sample(10)

,category,needs_met,feelings_guesses,needs_guesses,kws
kw_base,,,,,
neglect,fofeeling,na,"lonely, angry, scared","connection, inclusion, belonging, acceptance","[neglect, neglects, neglecting, neglected]"
rest,need,na,na,na,"[rest, rests, resting, rested]"
jittery,feeling,no,na,na,"[jittery, jitteries, jitterying, jitteried]"
miserable,feeling,no,na,na,"[miserable, miserables, miserabling, miserabled]"
outrage,feeling,no,na,na,"[outrage, outrages, outraging, outraged]"
irk,feeling,no,na,na,"[irk, irks, irking, irked]"
authenticity,need,na,na,na,"[authenticity, authenticities, authenticitying..."
upset,feeling,no,na,na,"[upset, upsets, upsetting]"
water,need,na,na,na,"[water, waters, watering, watered]"


Then, load the datafile that contains the nvc key phrases

In [10]:
# Load json datafile
kp_data = pd.read_csv("data/kp_datafile.csv", index_col='kp')

# Lowercase everything
kp_data.index = kp_data.index.str.lower()
kp_data.apply(lambda x: x.str.lower(), axis=1)

# Check for duplicates - TODO

kp_data.sample(3)

,category,feedback,needs_met,feelings_guesses,needs_guesses
kp,,,,,
i need you to,thought,our needs are our own; they exist within us an...,NaN,NaN,NaN
you really need to,NaN,NaN,NaN,NaN,NaN
to be known,need,na,na,na,na


In [11]:
kp_data['kps'] = [en.lexeme(word) for word in kp_data.index]

In [12]:
kp_data['kp_base'] = [en.lemma(word) for word in kp_data.index]
kp_data = kp_data.set_index('kp_base')

In [13]:
kp_data.sample(5)

,category,feedback,needs_met,feelings_guesses,needs_guesses,kps
kp_base,,,,,,
feel like,evaluation,something in me feels,na,na,na,"[feel like, feel likes, feel liking, feel liked]"
like that,NaN,NaN,NaN,NaN,NaN,"[like that, like thats, like thatting, like th..."
you really need to,NaN,NaN,NaN,NaN,NaN,"[you really need to, you really need tos, you ..."
to be seen,need,na,na,na,na,"[to be seen, to be seens, to be seening, to be..."
to be heard,need,na,na,na,na,"[to be heard, to be heards, to be hearding, to..."


Okay! Now our datafiles are loaded, cleaned up and we can get started.

# 👉 Attempt 1: NVC Feedback from Syntactical Analysis

Using syntactical analysis we can extract the basic information from a user's statement (ie the subject, the feeling/fofeeling/need, and also sometimes provide feelings/needs guesses in the case of a fofeeling). Let's see how far we can take syntactical analysis in helping us get feedback about our input statement.

**Objective** - this part is about taking potentially non-NVC input sentences and giving feedback (if needed) to help guide a user through turning them into NVC sentences. Hopefully we can do this using keywords/phrases data as well as looking at parts of speech.

## Using POS Tags, Keywords and Phrases

**Objective** Categorize feeling, needs, evaluations, fofeelings, absolutes, etc. based on data provided about common keywords and phrases

**Examples**

If the user says: *You manipulated me!* The system can tell us:
* 'manipulated' is a fofeeling
* provide somefeelings guesses

If the user says: *I am in a peaceful state.* The system can tell us:
* 'peacefulness' is a feeling
* there are no unmet needs

If the user says: *I would like some empathy.* The system can tell us:
* this is a need request for 'empathy'

Try typing an input statement into the text box after running the cell below. The input statement can be NVC language or not NVC lanauge. The point is it should be something that someone says to another person when trying to express how they feel.

(Ex. "I hate it when you slam the door like that!" or "You make me feel ugly" or "I feel like you manipulate all the time" or "I need you to speak softly")

In [44]:
# Type an input statement
sent=input()
sent = sent.lower()

 I am feeling compassionate


Let's do some pre-processing on the sentence you input

In [45]:
doc = nlp(sent)
displacy.render(doc, style="dep", jupyter=True)

In [46]:
def get_res(doc):
    words = [tok.text for tok in doc]
    data = [[tok.lemma_, ps.stem(tok.text), tok.pos_, tok.tag_, tok.dep_] for tok in doc]
    res = pd.DataFrame(index=words, data=data, columns=['lemma', 'stem', 'pos', 'tag', 'dep'])
    return res

res = get_res(doc)
res

,lemma,stem,pos,tag,dep
i,i,i,PRON,PRP,nsubj
am,be,am,AUX,VBP,aux
feeling,feel,feel,VERB,VBG,ROOT
compassionate,compassionate,compassion,ADJ,JJ,acomp


Check for adverbs and adjectives because often these are evaluations (descriptors) rather than feelings, needs, or observations.

**POS Tag Key**

| TAG  | POS                                                                      | Examples           |
|------|--------------------------------------------------------------------------|--------------------|
| CC   | coordinating conjunction                                                 |                    |
| CD   | cardinal digit                                                           |                    |
| DT   | determiner                                                               |                    |
| EX   | existential there (like: "there is" ... think of it like "there exists") |                    |
| FW   | foreign word                                                             |                    |
| IN   | preposition/subordinating conjunction                                    |                    |
| **JJ**   | adjective                                                                | 'big'              |
| **JJR**  | adjective, comparative                                                   | 'bigger'           |
| **JJS**  | adjective, superlative                                                   | 'biggest'          |
| LS   | list marker                                                              | 1)                 |
| MD   | modal                                                                    | could, will        |
| NN   | noun, singular 'desk'                                                    |                    |
| NNS  | noun plural                                                              | 'desks'            |
| NNP  | proper noun, singular                                                    | 'Harrison'         |
| NNPS | proper noun, plural                                                      | 'Americans'        |
| PDT  | predeterminer                                                            | 'all the kids'     |
| POS  | possessive ending                                                        | parent\'s          |
| PRP  | personal pronoun                                                         | I, he, she         |
| PRP  | possessive pronoun                                                       | my, his, hers      |
| **RB**   | adverb                                                                   | very, silently,    |
| **RBR**  | adverb, comparative                                                      | better             |
| **RBS**  | adverb, superlative                                                      | best               |
| RP   | particle                                                                 | give up            |
| TO   | to                                                                       | go 'to' the store. |
| UH   | interjection                                                             | errrrrrrrm         |
| VB   | verb, base form                                                          | take               |
| VBD  | verb, past tense                                                         | took               |
| VBG  | verb, gerund/present participle                                          | taking             |
| VBN  | verb, past participle                                                    | taken              |
| VBP  | verb, sing. present, non-3d                                              | take               |
| VBZ  | verb, 3rd person sing. present                                           | takes              |
| WDT  | wh-determiner                                                            | which              |
| WP   | wh-pronoun                                                               | who, what          |
| WP$  | possessive wh-pronoun                                                    | whose              |
| WRB  | wh-abverb                                                                | where, when        |

*Src: https://pythonprogramming.net/part-of-speech-tagging-nltk-tutorial/*

In [47]:
r = re.compile(r'(JJ.*)|(RB.*)')
pos = res.loc[res.tag.apply(lambda x: bool(r.match(x))) == True]
pos

,lemma,stem,pos,tag,dep
compassionate,compassionate,compassion,ADJ,JJ,acomp


Check for words ending in "ly", these tend to be evaluations also (softly, annoyingly, etc). We could try using this in conjunction with looking for the specific parts of speech. Maybe if could give us a better result sometimes.

In [50]:
r = re.compile(r'.*ly')
lys = res.loc[res.tag.apply(lambda x: bool(r.match(x))) == True]
lys

,lemma,stem,pos,tag,dep


Gather the resulting descriptors from above and save them for later

In [51]:
# Resulting descriptors found
desc = pd.concat([pos,lys])
desc

,lemma,stem,pos,tag,dep
compassionate,compassionate,compassion,ADJ,JJ,acomp


Next, we can check the input for specific keywords and phrases.

In [52]:
# Check if any common NVC phrases (from our datafile) were used in the sentence
kp_matches = pd.DataFrame()
for kp_base in kp_data.index:
    if kp_base in sent:
        kp_matches = kp_matches.append(kp_data.loc[kp_base])
kp_matches.head()

,category,feedback,feelings_guesses,kps,needs_guesses,needs_met
i am,evaluation,you are not defined by what you feel; try inst...,na,"[i am, i ams, i amming, i ammed]",na,na


In [53]:
# Check if any common NVC keywords (from our datafile) were used in the sentence
kw_matches = pd.DataFrame()
for w in res.index:
    w_lemma = en.lemma(w)
    if w_lemma in kw_data.index:
        kw_matches = kw_matches.append(kw_data.loc[w_lemma])
kw_matches.head()

,category,feelings_guesses,kws,needs_guesses,needs_met
compassionate,feeling,na,"[compassionate, compassionates, compassionatin...",na,yes


In [54]:
matches = pd.concat([kp_matches,kw_matches])
matches

,category,feedback,feelings_guesses,kps,needs_guesses,needs_met,kws
i am,evaluation,you are not defined by what you feel; try inst...,na,"[i am, i ams, i amming, i ammed]",na,na,NaN
compassionate,feeling,NaN,na,NaN,na,yes,"[compassionate, compassionates, compassionatin..."


Let's combine the data we acquired from all the steps above to see what kind of feedback we can give the user about their input.

In [55]:
class Categories(Enum):
    FEELING = 'feeling'
    FOFEELING = 'fofeeling'
    NEED = 'need'
    THOUGHT = 'thought'
    
def build_fb_sentence(sent, matches, descriptors):
    res = ['Okay, thanks for sharing.']
    
    if not descriptors.empty:
        res += ['I noticed that you used what I think may be one or more descriptors: {0}.'.format(','.join(descriptors.index.tolist()))]
        res += [f'Generally, descriptors are evaluations or thoughts and not observations, feelings or needs.']

    if not matches.empty:
        for kw in matches.index:
            if bool(matches.loc[kw].category == Categories.THOUGHT.value):
                res += [f'The phrase {kw} is a thought, not a feeling or need']
            elif matches.loc[kw].category == Categories.FEELING.value:
                needs = 'DO NOT' if matches.loc[kw].needs_met == 'yes' else ''
                res += [f'From what I understand, you are feeling {kw} and {needs} HAVE unmet needs']
            elif matches.loc[kw].category == Categories.FOFEELING.value:
                f_guesses = matches.loc[kw].feelings_guesses
                n_guesses = matches.loc[kw].needs_guesses
                res += [f'{kw} is a fofeeling, perhaps you are FEELING {f_guesses} and have unmet needs like {n_guesses}']
            elif matches.loc[kw].category == Categories.NEED.value:
                res += [f'I understand that you have the need: {kw}']
    else:
        res += [f'Im not sure if you are expressing a feeling, thought, need, or fofeeling.']
    
    return res

res = build_fb_sentence(res.index, matches, desc)
print(res)

# Play result
engine.say(str().join(res))
engine.runAndWait()

['Okay, thanks for sharing.', 'I noticed that you used what I think may be one or more descriptors: compassionate.', 'Generally, descriptors are evaluations or thoughts and not observations, feelings or needs.', 'From what I understand, you are feeling compassionate and DO NOT HAVE unmet needs']


## Conclusion and Discussion

If we look at the part of speech and also key words/phrases we can provide some reasonably information rich feedback about the users input sentence. Next, I'd like to run a better `get_nvc_feedback()` function for a variety of user input sentences and see how it does. 

I'll create an nvc_toolkit module for this.